# [--- Day 5: If You Give A Seed A Fertilizer ---](https://adventofcode.com/2023/day/5)

* Themes: Intervals, functional, mapping

## Setup

Use the `input` file if present, otherwise use the sample input.

In [1]:
from pprint import pprint
import numpy as np
from matplotlib import pyplot as plt

verbose = True
is_sample = False

try:
  input = open("input", "r").read().splitlines()
except FileNotFoundError:
  input = """seeds: 79 14 55 13

seed-to-soil map:
50 98 2
52 50 48

soil-to-fertilizer map:
0 15 37
37 52 2
39 0 15

fertilizer-to-water map:
49 53 8
0 11 42
42 0 7
57 7 4

water-to-light map:
88 18 7
18 25 70

light-to-temperature map:
45 77 23
81 45 19
68 64 13

temperature-to-humidity map:
0 69 1
1 0 69

humidity-to-location map:
60 56 37
56 93 4"""
  input = input.split("\n")
  verbose = True
  is_sample = True

if verbose:
  pprint(input)



['seeds: 79 14 55 13',
 '',
 'seed-to-soil map:',
 '50 98 2',
 '52 50 48',
 '',
 'soil-to-fertilizer map:',
 '0 15 37',
 '37 52 2',
 '39 0 15',
 '',
 'fertilizer-to-water map:',
 '49 53 8',
 '0 11 42',
 '42 0 7',
 '57 7 4',
 '',
 'water-to-light map:',
 '88 18 7',
 '18 25 70',
 '',
 'light-to-temperature map:',
 '45 77 23',
 '81 45 19',
 '68 64 13',
 '',
 'temperature-to-humidity map:',
 '0 69 1',
 '1 0 69',
 '',
 'humidity-to-location map:',
 '60 56 37',
 '56 93 4']


## Problem specific functions and classes



In [2]:
def get_map(section):
  """ Parse out the map details as a list of lists """
  res = []
  collect = False
  for line in input:
    if line == section:
      collect = True
      continue
    if line == "" and collect:
      return res
    if collect:
      res.append([int(i) for i in line.split()])
  return res

seeds = [int(i) for i in input[0].split(":")[1].split()]
print(seeds)

maps = []
maps.append(get_map("seed-to-soil map:"))
maps.append(get_map("soil-to-fertilizer map:"))
maps.append(get_map("fertilizer-to-water map:"))
maps.append(get_map("water-to-light map:"))
maps.append(get_map("light-to-temperature map:"))
maps.append(get_map("temperature-to-humidity map:"))
maps.append(get_map("humidity-to-location map:"))

pprint(maps)

[79, 14, 55, 13]
[[[50, 98, 2], [52, 50, 48]],
 [[0, 15, 37], [37, 52, 2], [39, 0, 15]],
 [[49, 53, 8], [0, 11, 42], [42, 0, 7], [57, 7, 4]],
 [[88, 18, 7], [18, 25, 70]],
 [[45, 77, 23], [81, 45, 19], [68, 64, 13]],
 [[0, 69, 1], [1, 0, 69]],
 [[60, 56, 37], [56, 93, 4]]]


In [3]:
def map_pos(map, position: int):
  for m in map:
    if m[1] <= position <= m[1] + m[2] - 1:
      return m[0] + position - m[1]
  return position

assert(map_pos(maps[0], 98) == 50)
assert(map_pos(maps[0], 99) == 51)
assert(map_pos(maps[0], 100) == 100)

## Part 1


In [4]:
res = []
for seed in seeds:
  for map in maps:
    seed = map_pos(map, seed)
  res.append(seed)

closest = min(res)

if is_sample:
  assert(closest == 35)

## Part 2

In [5]:
def get_seed_intervals(seeds):
  """ Converts original seed input to [x, y) intervals """
  seed_intervals = []
  for i in range(0, len(seeds), 2):
    seed_intervals.append([seeds[i], seeds[i] + seeds[i + 1]])
  return seed_intervals

assert(
    get_seed_intervals([79, 14, 55, 13]) == [[79, 93], [55, 68]]
)

In [6]:
def map_to_interval(m):
  """ Converts input mapping to interval format """
  return [m[1], m[1] + m[2]], [m[0], m[0] + m[2]]

map_to_interval([50, 98, 2])

([98, 100], [50, 52])

In [7]:
def del_from_interval(interval, to_del):
  """ Returns a tuple of 2 elements, first is a list of zero, one or 2
  remaining intervals and the second being extracted interval """

  if (
    to_del[1] <= interval[0] or
    to_del[0] >= interval[1]
  ):
    return [interval], [] # Nothing deleted

  if (
    to_del[0] <= interval[0] and
    to_del[1] >= interval[1]
  ):
    return [], interval # Entire interval deleted

  if (
    to_del[0] >= interval[0] and
    to_del[1] <= interval[1]
  ):
    inres = []
    if interval[0] != to_del[0]:
      inres.append([interval[0], to_del[0]])
    if to_del[1] != interval[1]:
      inres.append([to_del[1], interval[1]])
    return inres, to_del

  if (
    to_del[0] <= interval[0]
  ):
    return [[to_del[1], interval[1]]], [interval[0], to_del[1]]

  return [[interval[0], to_del[0]]], [to_del[0], interval[1]]

assert(del_from_interval([5, 7], [4, 5]) == ([[5, 7]], []))
assert(del_from_interval([5, 7], [7, 8]) == ([[5, 7]], []))
assert(del_from_interval([5, 7], [4, 6]) == ([[6, 7]], [5, 6]))
assert(del_from_interval([5, 7], [6, 9]) == ([[5, 6]], [6, 7]))
assert(del_from_interval([1, 9], [4, 7]) == ([[1, 4], [7, 9]], [4, 7]))

In [8]:


def delete_from_seeds(intervals, m):
  new_intervals = []
  deleted = []
  for interval in intervals:
    delete_res = del_from_interval(interval, m)
    new_intervals += delete_res[0]
    if delete_res[1]:
      deleted.append(delete_res[1])

  return new_intervals, deleted

print(delete_from_seeds([[5, 8], [8, 10]], [7, 9]))

assert(
  delete_from_seeds([[5,8]], [5, 6]) == ([[6, 8]], [[5, 6]])
)
assert(
  delete_from_seeds([[5, 8], [8, 10]], [7, 9]) == ([[5, 7], [9, 10]], [[7, 8], [8, 9]])
)

([[5, 7], [9, 10]], [[7, 8], [8, 9]])


In [9]:
seed_intervals = get_seed_intervals(seeds)
print("seed_intervals", seed_intervals)

for map in maps:
  to_move = []

  for m in map:
    from_to = map_to_interval(m)
    move_dir = from_to[1][0] - from_to[0][0]
    seed_intervals, deleted = delete_from_seeds(seed_intervals, from_to[0])
    for d in deleted:
      d[0] += move_dir
      d[1] += move_dir
      to_move.append(d)

  for d in to_move:
    seed_intervals.append(d)

starting = []
for seed in seed_intervals:
  starting.append(seed[0])

closest = min(starting)
print(closest)

if is_sample:
  assert(closest == 46)



seed_intervals [[79, 93], [55, 68]]
46
